In [ ]:
import numpy as np
import cupy as cp

In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

 #####  ``nvidia-smi``  is a command we can use to get the statistics GPU devices available in a Node

In [ ]:
!nvidia-smi

We can do the same using CuPy functions

In [ ]:
ngpus = cp.cuda.runtime.getDeviceCount()
print('#GPU = ' + str(ngpus))

In [ ]:
for g in range(ngpus):
    print(cp.cuda.runtime.getDeviceProperties(g))


##### CuPy has a concept of a current device, which is the default GPU device on which on which all operation of related to CuPy takes place. Unless specifically mentioned, all operation taskes place in this default device. 

In [ ]:
print('Current Device = ' + str(cp.cuda.runtime.getDevice()))

# NDARRAY

##### CuPy implements a subset of Numpy interface. 

``cupy.ndarray `` is akin to the ``numpy.ndarray ``. It is an array object that represents a multidimensional, homogeneous array of fixed-size items. This is the core of CuPy


##### A call to the ``numpy.array()`` allocates the data in the main memory

In [ ]:
x_cpu = np.array([1, 2, 3]) # allocate an ndarray in the main memory

##### While a call to the ``cupy.array()`` allocates the data in the GPU memory. If no device is specified the memory gets allocated in the ``current`` device.

In [ ]:
x_gpu = cp.array([1, 2, 3])

##### We can use the  device context manage to switch between the devices

In [ ]:
with cp.cuda.Device(1):  # alloacte an ndarray in the gpu memory of the 1st device
    x_on_gpu1 = cp.array([1, 2, 3, 4, 5])

In [ ]:
print('location of x_gpu = ' + str(x_gpu.device) )
print('location of x_gpu1 = ' + str(x_on_gpu1.device) )

##### CuPy always assumes that the operations are performed on the currently active device. 

# Data Transfers

In [ ]:
x_cpu = np.array([1, 2, 3])

In a normal CUDA workflow we have to allocate the memory on the GPU and the move the data to the GPU memory. In CuPy this is not required, the memory allocation and data movement can be done in a single operation.

In [ ]:
x_gpu_0 = cp.asarray(x_cpu)  # move the ndarray from host mem to GPU0 memeory.

In the past any communication between two GPUs had to go throgh the PCIe card. But now NVIDIA offeres a technology called NVLink. NVLink is a direct GPU-to-GPU interconnect that scales multi-GPU input/output (IO) within a node. This makes GPU-to-GPU transfer (D2D tranfer) much faster than GPU-to-Host (D2H transfer) or Host-to-GPU transfer (H2D transfer). 

In [ ]:

with cp.cuda.Device(1):
    x_gpu_1 = cp.asarray(x_gpu_0)  # move the ndarray to GPU0 to GPU1.

##### There are two ways to fetch the data from the GPU to host ``cupy.ndarray.get()`` or ``cupy.asnumpy``

In [ ]:
with cp.cuda.Device(0):
    x_cpu = cp.asnumpy(x_gpu_0)  # move the array from GPU 0 back to the host.

In [ ]:
type(x_cpu)

In [ ]:
with cp.cuda.Device(1):
    x_cpu = x_gpu_1.get()  # move the array from GPU 1 back to the host.

In [ ]:
type(x_cpu)

# Device Agnostic Codes

In [ ]:
x_cpu = np.array([1, 2, 3])
x_gpu = cp.array([1, 2, 3, 4, 5])

 As cupy mimicks numpy we can build device agnostics codes. That is, we can make function calls to a data, without the knowledge of where they reside. The ``cupy.get_array_module()`` function in CuPy returns a reference to cupy if any of its arguments resides on a GPU and numpy otherwise.

In [ ]:
def log_array(x):
    xp = cp.get_array_module(x)  # cupy ndarray array reference is returned if x is in GPU
                                 # numpy ndarray array reference is returned if x is in host memory
    xp.log1p(xp.exp(-abs(x))) 

In [ ]:
log_array(x_cpu) # the function works with numpy ndarray

In [ ]:
log_array(x_gpu) # the function works with cupy ndarray

# Explicit Data Transfer

In [ ]:
x_cpu = np.array([1, 2, 3])
y_cpu = np.array([4, 5, 6])

In [ ]:
z_cpu = x_cpu + y_cpu

While CuPy automates most data transfers, we may need to move things explicitly between host and GPU, depdending on the application we are implementing.  CuPy provides two methods ``cupy.asnumpy()`` and ``cupy.asarray()`` to do this. 

The ``cupy.asnumpy() `` method returns a NumPy array (array on the host), whereas ``cupy.asarray()`` method returns a CuPy array (array on the current device)

In [ ]:
x_gpu = cp.asarray(x_cpu) #copy host data to GPU mem

In [ ]:
# x_gpu is located in device memory
# y_cpu is located in host memory
# z_cpu is located in host memory

z_cpu = x_gpu + y_cpu # generats an error


We cant have an opeartion where on memory is located on on memory domain (Host) and the other is located on another memory domain (GPU). 

In [ ]:
x_gpu = cp.asarray(x_cpu) #copy host data to GPU mem

# x_gpu is located in device memory
# y_cpu is located in host memory
# z_cpu is located in host memory



#### So either both the data we operate on must be in host memory

In [ ]:
# move x_gpu to host memory
x_cpu = cp.asnumpy(x_gpu) 
z_cpu = x_cpu + y_cpu



#### .... or on the GPU memory

In [ ]:
# or move y_cpu to device memory
y_gpu = cp.asarray(y_cpu)
z_gpu = x_gpu + y_gpu

# User defined kernels

Kernels are functions that will be run on the GPU. CuPy provides easy ways to define three types of CUDA kernels: elementwise kernels, reduction kernels and raw kernels

## Elementwise Kernels

An element wise kerenel has 4 parts :
1. an input argument list:
2. an output argument list
3. loop body : defines the operation on each element of the argument
4. kernel name


In [ ]:
element_diff = cp.ElementwiseKernel('float32 x, float32 y', 
                                    'float32 z', 
                                    'z = (x - y)', 
                                    'element_diff')



In [ ]:
x = cp.array([1, 2, 3], dtype=np.float32)
y = cp.array([4, 5, 6], dtype=np.float32)

In [ ]:
z = element_diff(x, y)
print(z)

In [ ]:
z = cp.empty((1, 3), dtype=np.float32)
element_diff(x, y, z)
print(z)

#### What happens when the ndarray is of different dimension?

#### Type can be inferred from the arguments

In [ ]:
element_diff = cp.ElementwiseKernel('T x, T y', 
                                    'T z', 'z = (x - y)', 
                                    'element_diff')

In [ ]:
x = cp.array([1, 2, 3], dtype=np.float32)
y = cp.array([4, 5, 6], dtype=np.float32)

z = element_diff(x, y)
print(z)



In [ ]:
x = cp.array([1, 2, 3], dtype=np.int32)
y = cp.array([4, 5, 6], dtype=np.int32)

z = element_diff(x, y)
print(z)

We can also write a kernel with manual indexing for some arguments by telling ElementwiseKernel class to use manual indexing. This is done by adding the ``raw`` keyword preceding the type specifier

In [ ]:
# i indicates the index within the loop
# _ind.size() indicates total number of elements to apply the elementwise operation

element_sum = cp.ElementwiseKernel('T x, raw T y', 
                                   'T z', 'z = x + y[_ind.size() - i - 1]', 
                                   'element_sum')


In [ ]:
x = cp.array([1, 2, 3], dtype=np.float32)
y = cp.array([4, 5, 6], dtype=np.float32)

z = element_sum(x, y)
print(z)

## Reduction kernels

A reduction kernel has 4 parts :
1. Identity value: This value is used for the initial value of reduction.
2. Mapping expression: It is used for the pre-processing of each element to be reduced.
3. Reduction expression: It is an operator to reduce the multiple mapped values. The special variables a and b are used for its operands.
4. Post mapping expression: It is used to transform the resulting reduced values. The special variable a is used as its input. Output should be written to the output parameter.

In [ ]:
reduction_kernel = cp.ReductionKernel(
    'T x', # input param
    'T y',  # output param
    'x',  # map
    'a + b',  # reduce
    'y = a',  # post-reduction map
    '0',  # identity value
    'reduction_kernel'  # kernel name
)

In [ ]:
x = cp.arange(10, dtype=np.float32).reshape(1, 10)
print(x)

In [ ]:
reduction_kernel(x, axis=1)

In [ ]:
x = cp.arange(10, dtype=np.float32).reshape(2, 5)
print(x)

In [ ]:
reduction_kernel(x, axis=1)

#### TODO: Change the above function to find the sum of squares of each element in the array
That is, If the array is [2, 3, 4, 5], find 2^2 + 3^2 + 4^2 + 5^2 

## Raw kernels

In [ ]:
add_kernel = cp.RawKernel(r'''
    extern "C" __global__
    void my_add(const float* x1, const float* x2, float* y) 
    {
        int tid = blockDim.x * blockIdx.x + threadIdx.x;
        y[tid] = x1[tid] + x2[tid];
    }''', 'my_add')



In [ ]:
x = cp.arange(25, dtype=cp.float32).reshape(5, 5)
y = cp.arange(25, dtype=cp.float32).reshape(5, 5)
z = cp.zeros((5, 5), dtype=cp.float32)



In [ ]:
# When calling a raw kernel ypu have to specify  
# how threads are grouped (grids and blocks)
# https://developer.nvidia.com/blog/cuda-refresher-cuda-programming-model/

add_kernel((5,), (5,), (x, y, z))
print(z)

# CUDA Events

In [ ]:
a_cp = cp.arange(10)
b_cp = cp.arange(10)


In [ ]:
e1 = cp.cuda.Event() # create an event

In [ ]:
e1.record() # Records an event on the stream.

In [ ]:
a_cp = b_cp * a_cp + 8

In [ ]:
e2 = cp.cuda.get_current_stream().record() # create and record the event

In [ ]:
s = cp.cuda.Stream.null
# make sure the stream wait for the second event
s.wait_event(e2)



In [ ]:
with s:
    # all actions in a stream happens seuentially
    # as the stream is waiting for the second event to complete
    # we can be assured that all the operations before it also has been complete.
    a_np = cp.asnumpy(a_cp)

# Waits for the stream that track an event to complete that event
e2.synchronize()
t = cp.cuda.get_elapsed_time(e1, e2)

print(t)


In [ ]:
# https://developer.nvidia.com/blog/how-implement-performance-metrics-cuda-cc/

# CUDA Streams

In [ ]:
a_np = np.arange(10)
s = cp.cuda.Stream()



In [ ]:
with s:
   a_cp = cp.asarray(a_np)  # H2D transfer on stream s
   b_cp = cp.sum(a_cp)      # kernel launched on stream s 



In [ ]:
# or we can use 'use()'
# if we use 'use() any subsequent CUDA operation will completed
# using the stream we specify, until we make a change 
s.use()

b_np = cp.asnumpy(b_cp)


In [ ]:
assert s == cp.cuda.get_current_stream()

In [ ]:
# go back to the default stream
cp.cuda.Stream.null.use()

In [ ]:
assert s == cp.cuda.get_current_stream() # run fails is assert condition is false
                                         # generates an error 
